In [1]:
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as ipw

from lib.dataset_wrapper import Dataset
from lib import abx_utils
from lib import notebooks

In [2]:
DATASET_NAME = "pb2007"

dataset = Dataset(DATASET_NAME)
assert dataset.has_palate
items_ema = dataset.get_items_data("ema")

In [3]:
LIPS_CONSONANTS = ["p", "b"]
TONGUE_CONSONANTS = ["t", "d", "k", "g"]
TESTED_CONSONANTS = dataset.phones_infos["consonants"]
CONSONANTS_COIL = {
    "t": slice(2, 4),
    "s": slice(2, 4),
    "d": slice(2, 4),
    "z": slice(2, 4),
    "n": slice(2, 4),
    "k": slice(4, 6),
    "s^": slice(4, 6),
    "g": slice(4, 6),
    "z^": slice(4, 6),
    "r": slice(4, 6),
    "l": slice(2, 4),
    "j": slice(4, 6),
}

vowels = dataset.phones_infos["vowels"]
dataset_lab = {DATASET_NAME: dataset.lab}
palate = dataset.palate

In [4]:
consonants_indexes = abx_utils.get_datasets_phones_indexes(
    dataset_lab, TESTED_CONSONANTS, vowels
)

In [5]:
def show_consonant(consonant):
    consonant_indexes = consonants_indexes[consonant]
    
    def show_occurence(i_index):
        index = consonant_indexes[i_index]
        print(index)
        item_ema = items_ema[index[1]]
        consonant_ema = item_ema[index[2]]
        
        consonant_coil = CONSONANTS_COIL[consonant]
        nb_frames = len(consonant_ema)
        
        coil_ema = consonant_ema[:, consonant_coil]
        repeated_palate = np.tile(palate, (nb_frames, 1, 1))
        
        coil_distances = np.sqrt(((coil_ema[:, None, :] - repeated_palate) ** 2).sum(axis=-1)).min(axis=1)
        occlusion_frames = (coil_distances < OCCLUSION_CEIL)
        occlusion_start = occlusion_frames.argmax()
        occlusion_end = nb_frames - occlusion_frames[::-1].argmax() - 1
        print(occlusion_start, occlusion_end)
        # if coil_distances[occlusion_start] >= OCCLUSION_CEIL: continue
        
#         consonant_ema_norm = consonant_ema - consonant_ema.mean(axis=0)
#         lower_lip_ema = consonant_ema_norm[:, 8:10]
#         upper_lip_ema = consonant_ema_norm[:, 10:12]
#         tongue_tip_ema = consonant_ema_norm[:, 2:4]
#         tongue_mid_ema = consonant_ema_norm[:, 4:6]
        
#         YLIM = 5
        
#         plt.figure()
        
#         ax = plt.subplot(421)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(upper_lip_ema[:, 0])
#         ax = plt.subplot(423)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(upper_lip_ema[:, 1])
#         ax = plt.subplot(425)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(lower_lip_ema[:, 0])
#         ax = plt.subplot(427)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(lower_lip_ema[:, 1])
        
        
#         ax = plt.subplot(422)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(tongue_tip_ema[:, 0])
#         ax = plt.subplot(424)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(tongue_tip_ema[:, 1])
#         ax = plt.subplot(426)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(tongue_mid_ema[:, 0])
#         ax = plt.subplot(428)
#         ax.set_ylim(-YLIM, YLIM)
#         ax.plot(tongue_mid_ema[:, 1])
        
#         plt.show()
        
        notebooks.show_ema(consonant_ema)
    
    ipw.interact(show_occurence, i_index=range(len(consonant_indexes)))

ipw.interactive(show_consonant, consonant=consonants_indexes.keys())

interactive(children=(Dropdown(description='consonant', options=('p', 'f', 'b', 'v', 'm', 't', 's', 'd', 'z', …